# Vision Transformer


## Lab adapted to ViT instead of CNN


## Mixup ago


In [9]:
!pip uninstall tqdm -y


  Using cached tqdm-4.66.2-py3-none-any.whl.metadata (57 kB)
Using cached tqdm-4.66.2-py3-none-any.whl (78 kB)
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# #!/usr/bin/env python3 -u
# # Copyright (c) 2017-present, Facebook, Inc.
# # All rights reserved.
# #
# # This source code is licensed under the license found in the LICENSE file in
# # the root directory of this source tree.
# from __future__ import print_function

# import argparse
# import csv
# import os
# from torchvision.transforms import Resize

# import numpy as np
# import torch
# from torch.autograd import Variable
# import torch.backends.cudnn as cudnn
# import torch.nn as nn
# import torch.optim as optim
# import torchvision.transforms as transforms
# import torchvision.datasets as datasets


# import models


# import argparse


# def parse_args():
#     parser = argparse.ArgumentParser(description="PyTorch CIFAR10 Training with ViT")
#     parser.add_argument("--lr", default=0.1, type=float, help="learning rate")
#     parser.add_argument(
#         "--resume", "-r", action="store_true", help="resume from checkpoint"
#     )
#     parser.add_argument(
#         "--model",
#         default="vit_small_patch16_224",
#         type=str,
#         help="model type (default: vit_small_patch16_224)",
#     )
#     parser.add_argument("--name", default="0", type=str, help="name of run")
#     parser.add_argument("--seed", default=0, type=int, help="random seed")
#     parser.add_argument("--batch-size", default=128, type=int, help="batch size")
#     parser.add_argument("--epoch", default=200, type=int, help="total epochs to run")
#     parser.add_argument(
#         "--no-augment",
#         dest="augment",
#         action="store_false",
#         help="use standard augmentation (default: True)",
#     )
#     parser.add_argument("--decay", default=1e-4, type=float, help="weight decay")
#     parser.add_argument(
#         "--alpha",
#         default=1.0,
#         type=float,
#         help="mixup interpolation coefficient (default: 1)",
#     )

#     # Use parse_known_args to ignore unrecognized command line arguments
#     args, _ = parser.parse_known_args()
#     return args


# # Use this function where you initialize your training setup
# args = parse_args()


# use_cuda = torch.cuda.is_available()

# best_acc = 0  # best test accuracy
# start_epoch = 0  # start from epoch 0 or last checkpoint epoch

# if args.seed != 0:
#     torch.manual_seed(args.seed)

# # Data
# print("==> Preparing data..")
# if args.augment:
#     transform_train = transforms.Compose([
#         Resize((224, 224)),  # Resize images to 224x224 pixels
#         transforms.RandomCrop(32, padding=4),
#         transforms.RandomHorizontalFlip(),
#         transforms.ToTensor(),
#         transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
#     ])
# else:
#     transform_train = transforms.Compose([
#         Resize((224, 224)),  # Resize images to 224x224 pixels
#         transforms.ToTensor(),
#         transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
#     ])

# transform_test = transforms.Compose([
#     Resize((224, 224)),  # Resize images to 224x224 pixels
#     transforms.ToTensor(),
#     transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
# ])


# trainset = datasets.CIFAR10(
#     root="~/data", train=True, download=False, transform=transform_train
# )
# trainloader = torch.utils.data.DataLoader(
#     trainset, batch_size=args.batch_size, shuffle=True, num_workers=8
# )

# testset = datasets.CIFAR10(
#     root="~/data", train=False, download=False, transform=transform_test
# )
# testloader = torch.utils.data.DataLoader(
#     testset, batch_size=100, shuffle=False, num_workers=8
# )


# # Model
# if args.resume:
#     # Load checkpoint.
#     print("==> Resuming from checkpoint..")
#     assert os.path.isdir("checkpoint"), "Error: no checkpoint directory found!"
#     checkpoint = torch.load("./checkpoint/ckpt.t7" + args.name + "_" + str(args.seed))
#     net = checkpoint["net"]
#     best_acc = checkpoint["acc"]
#     start_epoch = checkpoint["epoch"] + 1
#     rng_state = checkpoint["rng_state"]
#     torch.set_rng_state(rng_state)
# else:
#     print("==> Building model..")
#     net = models.__dict__[args.model]()

# if not os.path.isdir("results"):
#     os.mkdir("results")
# logname = (
#     "results/log_"
#     + net.__class__.__name__
#     + "_"
#     + args.name
#     + "_"
#     + str(args.seed)
#     + ".csv"
# )

# if use_cuda:
#     net.cuda()
#     net = torch.nn.DataParallel(net)
#     print(torch.cuda.device_count())
#     cudnn.benchmark = True
#     print("Using CUDA..")

# criterion = nn.CrossEntropyLoss()
# optimizer = optim.SGD(
#     net.parameters(), lr=args.lr, momentum=0.9, weight_decay=args.decay
# )


# def mixup_data(x, y, alpha=1.0, use_cuda=True):
#     """Returns mixed inputs, pairs of targets, and lambda"""
#     if alpha > 0:
#         lam = np.random.beta(alpha, alpha)
#     else:
#         lam = 1

#     batch_size = x.size()[0]
#     if use_cuda:
#         index = torch.randperm(batch_size).cuda()
#     else:
#         index = torch.randperm(batch_size)

#     mixed_x = lam * x + (1 - lam) * x[index, :]
#     y_a, y_b = y, y[index]
#     return mixed_x, y_a, y_b, lam


# def mixup_criterion(criterion, pred, y_a, y_b, lam):
#     return lam * criterion(pred, y_a) + (1 - lam) * criterion(pred, y_b)


# def train(epoch):
#     print("\nEpoch: %d" % epoch)
#     net.train()
#     train_loss = 0
#     reg_loss = 0
#     correct = 0
#     total = 0
#     for batch_idx, (inputs, targets) in enumerate(trainloader):
#         if use_cuda:
#             inputs, targets = inputs.cuda(), targets.cuda()

#         inputs, targets_a, targets_b, lam = mixup_data(
#             inputs, targets, args.alpha, use_cuda
#         )
#         inputs, targets_a, targets_b = map(Variable, (inputs, targets_a, targets_b))
#         outputs = net(inputs)
#         loss = mixup_criterion(criterion, outputs, targets_a, targets_b, lam)
#         train_loss += loss.data[0]
#         _, predicted = torch.max(outputs.data, 1)
#         total += targets.size(0)
#         correct += (
#             lam * predicted.eq(targets_a.data).cpu().sum().float()
#             + (1 - lam) * predicted.eq(targets_b.data).cpu().sum().float()
#         )

#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()

#         print(
#             "Batch: %d/%d | Loss: %.3f | Reg: %.5f | Acc: %.3f%% (%d/%d)"
#             % (
#                 batch_idx + 1,
#                 len(trainloader),
#                 train_loss / (batch_idx + 1),
#                 reg_loss / (batch_idx + 1),
#                 100.0 * correct / total,
#                 correct,
#                 total,
#             )
#         )
#     return (train_loss / batch_idx, reg_loss / batch_idx, 100.0 * correct / total)


# def test(epoch):
#     global best_acc
#     net.eval()
#     test_loss = 0
#     correct = 0
#     total = 0
#     for batch_idx, (inputs, targets) in enumerate(testloader):
#         if use_cuda:
#             inputs, targets = inputs.cuda(), targets.cuda()
#         inputs, targets = Variable(inputs, volatile=True), Variable(targets)
#         outputs = net(inputs)
#         loss = criterion(outputs, targets)

#         test_loss += loss.data[0]
#         _, predicted = torch.max(outputs.data, 1)
#         total += targets.size(0)
#         correct += predicted.eq(targets.data).cpu().sum()

#         print("acc =", acc)
#         if epoch == start_epoch + args.epoch - 1 or acc > best_acc:
#             print("Saving..")
#         if acc > best_acc:
#             print("best_acc =", best_acc)
#         print("test_loss =", test_loss / batch_idx)
#         print("correct =", 100.0 * correct / total)
#     acc = 100.0 * correct / total
#     if epoch == start_epoch + args.epoch - 1 or acc > best_acc:
#         checkpoint(acc, epoch)
#     if acc > best_acc:
#         best_acc = acc
#     return (test_loss / batch_idx, 100.0 * correct / total)


# def checkpoint(acc, epoch):
#     # Save checkpoint.
#     print("Saving..")
#     state = {"net": net, "acc": acc, "epoch": epoch, "rng_state": torch.get_rng_state()}
#     if not os.path.isdir("checkpoint"):
#         os.mkdir("checkpoint")
#     torch.save(state, "./checkpoint/ckpt.t7" + args.name + "_" + str(args.seed))


# def adjust_learning_rate(optimizer, epoch):
#     """decrease the learning rate at 100 and 150 epoch"""
#     lr = args.lr
#     if epoch >= 100:
#         lr /= 10
#     if epoch >= 150:
#         lr /= 10
#     for param_group in optimizer.param_groups:
#         param_group["lr"] = lr


# if not os.path.exists(logname):
#     with open(logname, "w") as logfile:
#         logwriter = csv.writer(logfile, delimiter=",")
#         logwriter.writerow(
#             ["epoch", "train loss", "reg loss", "train acc", "test loss", "test acc"]
#         )

# for epoch in range(start_epoch, args.epoch):
#     train_loss, reg_loss, train_acc = train(epoch)
#     test_loss, test_acc = test(epoch)
#     adjust_learning_rate(optimizer, epoch)
#     with open(logname, "a") as logfile:
#         logwriter = csv.writer(logfile, delimiter=",")
#         logwriter.writerow(
#             [epoch, train_loss, reg_loss, train_acc, test_loss, test_acc]
#         )


ModuleNotFoundError: No module named 'models'

: 

In [7]:
from __future__ import print_function

import argparse
import csv
import os

import numpy as np
import torch
from torch.autograd import Variable
import torch.backends.cudnn as cudnn
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
from PIL import Image
from torchvision.transforms import Resize


import argparse


# -------------- ViT --------------
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.transformer = models.vit_b_16(pretrained=True)
        self.fc = nn.Linear(
            1000, 10
        )  # Assuming the output features from vit_b_32 are 1000

    def forward(self, x):
        x = self.transformer(x)
        x = self.fc(x)
        return x


# ------------- Mixup data -------------
def mixup_data(x, y, alpha=1.0, use_cuda=True):
    """Returns mixed inputs, pairs of targets, and lambda"""
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1

    batch_size = x.size()[0]
    if use_cuda:
        index = torch.randperm(batch_size).cuda()
    else:
        index = torch.randperm(batch_size)

    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam


# -------------- Mixup criterion --------------
def mixup_criterion(criterion, pred, y_a, y_b, lam):
    return lam * criterion(pred, y_a) + (1 - lam) * criterion(pred, y_b)


# -------------- Training --------------
if __name__ == "__main__":
    ## Prepare the CIFAR-10 dataset
    transform = transforms.Compose(
        [
            Resize((224, 224)),  # Resize images to 224x224 pixels
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
        ]
    )

    batch_size = 20
    trainset = torchvision.datasets.CIFAR10(
        root="./data", train=True, download=True, transform=transform
    )
    trainloader = torch.utils.data.DataLoader(
        trainset, batch_size=batch_size, shuffle=True, num_workers=2
    )
    classes = (
        "plane",
        "car",
        "bird",
        "cat",
        "deer",
        "dog",
        "frog",
        "horse",
        "ship",
        "truck",
    )

    # TODO: Generate example images for visualization
    dataiter = iter(trainloader)
    images, labels = next(dataiter)

    im = Image.fromarray(
        (torch.cat(images.split(1, 0), 3).squeeze() / 2 * 255 + 0.5 * 255)
        .permute(1, 2, 0)
        .numpy()
        .astype("uint8")
    )
    im.save("train_pt_images.jpg")
    print("train_pt_images.jpg saved.")
    print(
        "Ground truth labels:"
        + " ".join("%5s" % classes[labels[j]] for j in range(batch_size))
    )

    ## Initialize the ViT model
    net = Net()
    use_cuda = torch.cuda.is_available()
    if use_cuda:
        net = net.cuda()

    ## Set the loss function and optimizer
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

    ## Train the model with Mixup augmentation
    alpha = 1.0  # Mixup interpolation coefficient, adjust as needed
    for epoch in range(2):  # loop over the dataset multiple times

        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data
            if use_cuda:
                inputs, labels = inputs.cuda(), labels.cuda()

            # TODO: Apply Mixup augmentation
            inputs, targets_a, targets_b, lam = mixup_data(
                inputs, labels, alpha, use_cuda
            )
            inputs, targets_a, targets_b = map(Variable, (inputs, targets_a, targets_b))
            optimizer.zero_grad()
            outputs = net(inputs)
            loss = mixup_criterion(criterion, outputs, targets_a, targets_b, lam)

            # TODO: Perform backpropagation and optimization
            loss.backward()
            optimizer.step()

            # TODO: Log training progress
            running_loss += loss.item()
            if i % 200 == 199:  # print every 200 mini-batches
                print(f"[{epoch + 1}, {i + 1}] loss: {running_loss / 200:.3f}")
                running_loss = 0.0

    print("Training done.")

    # TODO: Save the trained model
    torch.save(net.state_dict(), "saved_model_with_mixup.pt")
    print("Model with mixup saved.")


Files already downloaded and verified
train_pt_images.jpg saved.
Ground truth labels:  car  bird plane  deer truck   cat   dog horse plane  deer   car truck   car   car   car horse  deer truck  frog  ship
[1, 200] loss: 1.373
[1, 400] loss: 1.102
[1, 600] loss: 1.035
[1, 800] loss: 1.055
[1, 1000] loss: 1.038
[1, 1200] loss: 1.033
[1, 1400] loss: 0.982
[1, 1600] loss: 0.980
[1, 1800] loss: 0.970
[1, 2000] loss: 0.953
[1, 2200] loss: 0.982
[1, 2400] loss: 0.952
[2, 200] loss: 0.887
[2, 400] loss: 0.912
[2, 600] loss: 0.940
[2, 800] loss: 0.970
[2, 1000] loss: 0.953
[2, 1200] loss: 0.908
[2, 1400] loss: 0.914
[2, 1600] loss: 0.888
[2, 1800] loss: 0.941
[2, 2000] loss: 0.895
[2, 2200] loss: 0.908
[2, 2400] loss: 0.875
Training done.
Model with mixup saved.


In [9]:
from __future__ import print_function

import argparse
import csv
import os

import numpy as np
import torch
from torch.autograd import Variable
import torch.backends.cudnn as cudnn
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch
import torch.nn.functional as F
import torchvision.models as models
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
from PIL import Image
from torchvision.transforms import Resize


# -------------- ViT --------------
class ViT(nn.Module):
    def __init__(self):
        super().__init__()
        self.transformer = models.vit_b_32(pretrained=True)
        self.fc = nn.Linear(
            1000, 10
        )  # Assuming the output features from vit_b_32 are 1000

    def forward(self, x):
        x = self.transformer(x)
        x = self.fc(x)
        return x


# -------------- MixUp --------------
class MixUp(object):
    def __init__(self, alpha=1.0, sampling_method=1, seed=42):
        self.alpha = alpha
        self.sampling_method = sampling_method
        self.seed = seed
        self.torch_rng = torch.manual_seed(seed)
        self.np_rng = np.random.seed(seed)
        self.use_cuda = torch.cuda.is_available()

    def __call__(self, x, y):
        if self.sampling_method == 1:
            lam = self.get_lambda_beta()
        else:
            lam = self.get_lambda_uniform()

        batch_size = x.size()[0]
        if self.use_cuda:
            index = torch.randperm(batch_size).cuda()
            x = x.cuda()
            y = y.cuda()
        else:
            index = torch.randperm(batch_size)

        mixed_x = lam * x + (1 - lam) * x[index, :]
        y_a, y_b = y, y[index]

        return mixed_x, y_a, y_b, lam

    def get_lambda_beta(self):
        return np.random.beta(self.alpha, self.alpha)

    def get_lambda_uniform(self):
        return np.random.uniform(0, 1)

    def visualize(self, dataloader, num_images=16):
        dataiter = iter(dataloader)
        images, labels = next(dataiter)

        mixed_images = []
        for i in range(num_images):
            mixed_x, _, _, _ = self.__call__(images, labels)
            mixed_images.append(mixed_x[i])

        montage = transforms.Resize((224, 224))(
            torchvision.utils.make_grid(torch.stack(mixed_images), nrow=4)
        )
        montage = montage.permute(1, 2, 0).numpy() * 255
        image = Image.fromarray(montage.astype("uint8"))
        image.save("mixup.png")
        print("Montage of augmented images saved to 'mixup.png'")


# ------------- Mixup criterion --------------
def mixup_criterion(criterion, pred, y_a, y_b, lam):
    return lam * criterion(pred, y_a) + (1 - lam) * criterion(pred, y_b)


# -------------- Training --------------
if __name__ == "__main__":
    ## Prepare the CIFAR-10 dataset
    transform = transforms.Compose(
        [
            Resize((224, 224)),  # Resize images to 224x224 pixels
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
        ]
    )

    batch_size = 20
    trainset = torchvision.datasets.CIFAR10(
        root="./data", train=True, download=True, transform=transform
    )
    trainloader = torch.utils.data.DataLoader(
        trainset, batch_size=batch_size, shuffle=True, num_workers=2
    )
    classes = (
        "plane",
        "car",
        "bird",
        "cat",
        "deer",
        "dog",
        "frog",
        "horse",
        "ship",
        "truck",
    )

    ## Initialize the ViT model
    net = ViT()
    use_cuda = torch.cuda.is_available()
    if use_cuda:
        net = net.cuda()

    ## Set the loss function and optimizer
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

    ## Initialize MixUp
    mixup = MixUp(alpha=1.0, sampling_method=1, seed=42)

    ## Train the model with Mixup augmentation
    alpha = 1.0  # Mixup interpolation coefficient, adjust as needed
    for epoch in range(2):  # loop over the dataset multiple times
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data

            # Apply Mixup augmentation
            inputs, targets_a, targets_b, lam = mixup(inputs, labels)
            inputs, targets_a, targets_b = map(Variable, (inputs, targets_a, targets_b))

            if use_cuda:
                inputs, targets_a, targets_b = (
                    inputs.cuda(),
                    targets_a.cuda(),
                    targets_b.cuda(),
                )

            optimizer.zero_grad()
            outputs = net(inputs)
            loss = mixup_criterion(criterion, outputs, targets_a, targets_b, lam)

            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            if i % 200 == 199:
                print(f"[{epoch + 1}, {i + 1}] loss: {running_loss / 200:.3f}")
                running_loss = 0.0

    print("Training done.")

    # Visualize augmented images
    mixup.visualize(trainloader)

    # Save the trained model
    torch.save(net.state_dict(), "saved_model_with_mixup.pt")
    print("Model with mixup saved.")


Files already downloaded and verified
[1, 200] loss: 1.327
[1, 400] loss: 1.081
[1, 600] loss: 1.050
[1, 800] loss: 1.012
[1, 1000] loss: 1.035
[1, 1200] loss: 1.026
[1, 1400] loss: 1.031
[1, 1600] loss: 1.012
[1, 1800] loss: 0.951
[1, 2000] loss: 0.982
[1, 2200] loss: 0.955
[1, 2400] loss: 0.978
[2, 200] loss: 0.912
[2, 400] loss: 0.919
[2, 600] loss: 0.966
[2, 800] loss: 0.961
[2, 1000] loss: 0.932
[2, 1200] loss: 0.907
[2, 1400] loss: 0.867
[2, 1600] loss: 0.898
[2, 1800] loss: 0.890
[2, 2000] loss: 0.905
[2, 2200] loss: 0.916
[2, 2400] loss: 0.897
Training done.
Montage of augmented images saved to 'mixup.png'
Model with mixup saved.
